In [16]:
import os
import zipfile

stream_name = 'twitter_stream2'


zf = zipfile.ZipFile("lambda.zip", "w")
zf.write('lambda.py')
zf.close()

In [17]:
import base64
import json

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event))
    for record in event['Records']:
        # Kinesis data is base64 encoded so decode here
        payload = base64.b64decode(record['kinesis']['data'])
        print("Decoded payload: " + payload)
    return 'Successfully processed {} records.'.format(len(event['Records']))

In [18]:
#!/usr/bin/env python

import time
import json
import boto3

kinesis = boto3.client('kinesis')
iam = boto3.client('iam')
l = boto3.client('lambda')

def create_stream(name):
    """ Create kinesis stream, and wait until it is active """
    if name not in [f for f in kinesis.list_streams()['StreamNames']]:
        print('Creating Kinesis stream %s' % (name))
        kinesis.create_stream(StreamName=name, ShardCount=1)
    else:
        print('Kinesis stream %s exists' % (name))
    while kinesis.describe_stream(StreamName=name)['StreamDescription']['StreamStatus'] == 'CREATING':
        time.sleep(2)
    return kinesis.describe_stream(StreamName=name)['StreamDescription']


def create_role(name, policies=None):
    """ Create a role with an optional inline policy """
    policydoc = {
        "Version": "2012-10-17",
        "Statement": [
            {"Effect": "Allow", "Principal": {"Service": ["lambda.amazonaws.com"]}, "Action": ["sts:AssumeRole"]},
        ]
    }
    roles = [r['RoleName'] for r in iam.list_roles()['Roles']]
    if name in roles:
        print('IAM role %s exists' % (name))
        role = iam.get_role(RoleName=name)['Role']
    else:
        print('Creating IAM role %s' % (name))
        role = iam.create_role(RoleName=name, AssumeRolePolicyDocument=json.dumps(policydoc))['Role']

    # attach managed policy
    if policies is not None:
        for p in policies:
            iam.attach_role_policy(RoleName=role['RoleName'], PolicyArn=p)
    return role


def create_function(name, zfile, lsize=512, timeout=10, update=False):
    """ Create, or update if exists, lambda function """
    role = create_role(name + '_lambda', policies=['arn:aws:iam::aws:policy/service-role/AWSLambdaKinesisExecutionRole'])
    time.sleep(10)
    with open(zfile, 'rb') as zipfile:
        if name in [f['FunctionName'] for f in l.list_functions()['Functions']]:
            if update:
                print('Updating %s lambda function code' % (name))
                return l.update_function_code(FunctionName=name, ZipFile=zipfile.read())
            else:
                print('Lambda function %s exists' % (name))
                for f in funcs:
                    if f['FunctionName'] == name:
                        lfunc = f
        else:
            print('Creating %s lambda function' % (name))
            lfunc = l.create_function(
                FunctionName=name,
                Runtime='python2.7',
                Role=role['Arn'],
                Handler='lambda.lambda_handler',
                Description='Example lambda function to ingest a Kinesis stream',
                Timeout=timeout,
                MemorySize=lsize,
                Publish=True,
                Code={'ZipFile': zipfile.read()},
            )
        lfunc['Role'] = role
        return lfunc

def create_mapping(name, stream):
    """ add a mapping to a stream """
    sources = l.list_event_source_mappings(FunctionName=name,
                                           EventSourceArn=stream['StreamARN'])['EventSourceMappings']
    if stream['StreamARN'] not in [s['EventSourceArn'] for s in sources]:
        source = l.create_event_source_mapping(FunctionName=name, EventSourceArn=stream['StreamARN'],
                                      StartingPosition='TRIM_HORIZON')
    else:
        for s in sources:
            source = s
    return source

In [19]:
name = 'alonzo'

# create kinesis stream
stream = create_stream(name)

# Create a lambda function
lfunc = create_function(name, 'lambda.zip', update=True)

# add mapping to kinesis stream
create_mapping(name, stream)

Kinesis stream alonzo exists
IAM role alonzo_lambda exists
Updating alonzo lambda function code


{'BatchSize': 100,
 'EventSourceArn': 'arn:aws:kinesis:us-east-1:714861692883:stream/alonzo',
 'FunctionArn': 'arn:aws:lambda:us-east-1:714861692883:function:alonzo',
 'LastModified': datetime.datetime(2017, 11, 6, 15, 48, tzinfo=tzlocal()),
 'LastProcessingResult': 'No records processed',
 'State': 'Enabled',
 'StateTransitionReason': 'User action',
 'UUID': '9cfe3181-99d7-4028-88ce-75619d93e1e1'}

In [20]:
# aws kinesis put-record --stream-name alonzo --data "{'0': 'the', '1': 'lambda', '2': 'calculus'}"

response = kinesis.put_record(
    StreamName='twitter',
    Data="{'0': 'the', '1': 'lambda', '2': 'calculus'}",
    PartitionKey='1'
#     ExplicitHashKey='string',
#     SequenceNumberForOrdering='string'
)